In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import pickle 
import seaborn as sns
import json
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn import svm
from matplotlib.colors import ListedColormap

**BELOW 2 CELLS SHOULD STRICTLY BE RUN ONLY IF YOU HAVE COLLECTED NEW TRAIN DATA**

In [ ]:
# Re-formatting JSON data recieced from sensor
def add_line_to_json_file(file_path):
    # Read the original JSON content
    with open(file_path, 'r') as file:
        content = file.read()

    # Add the new line at the beginning
    new_content = '{"dataBlock": [\n' + content
    new_content=new_content[:-11]
    new_content=new_content.replace('] }', '] },')
    new_content=new_content+"] } ]}"

    # Write the modified content back to the file
    with open(file_path, 'w') as file:
        file.write(new_content)

In [ ]:
# Enter the details asked to reformat the json data to right format
num=int(input("Please enter the number of items trained: "))
list = []
for i in range(num):
    item=input("Please enter the item you trained: ")
    file_path = f'/eNose-main/scripts/{item}.json'
    if not os.path.exists(file_path):
        print(f"Error: The file at {file_path} does not exist.")
    else:
        add_line_to_json_file(file_path)
        print(f"Successfully modified file {item}.json")
        list.append(item)


**IF ABOVE TWO CELLS ARE RUN THEN COMMENT THE LINE IN BELOW CELL- list=['air','sanitizer','isopropylAlcohol']**

**IT IS PRESENT IN LINE 3**

In [ ]:
# Enable intelliense
%config IPCompleter.greedy=True

# get label for the data
dir = '/eNose-main/scripts/'
csvFilename = "psec2-data.csv"
f = open(csvFilename, "w")

# Comment the below line if above 2 cells is run
list=['air','sanitizer','isopropylAlcohol']

header = "sensor_id,time,temperature,pressure,humidity,gas_resistance,smell"

f.write(header + '\n')

for item in range(len(list)):

    # convert collected json data into a csv file
    jsonFilename = dir + list[item] + '.json'

    parsed_json = ""
    with open(jsonFilename) as json_file:
      parsed_json = json.load(json_file)

    #print(parsed_json)
    i = 0
    for block in parsed_json['dataBlock']:
        #print(block)
        #break
        for data in block['datapoints']:
            #print(data)
            result = ''
            #print(range(len(data) - 1))
            for x in data:
                result += str(x) + ','
        
            result += list[item]
            #print(result)
            f.write(result + '\n')
    
f.close()

In [ ]:
# read csv file into dataFrame
dataset = pd.read_csv(csvFilename)
print(str(len(dataset)) + ' records')
print(dataset.head())

In [ ]:
# split dataset
# 'Temperature','Pressure','Relative Humidity','Resistance Gassensor','Label Tag'
X = dataset.loc[:,['humidity', 'gas_resistance']]
y = dataset.loc[:, ['smell']]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=32767, test_size=0.05)


In [ ]:
print(len(X_train))
print('-'*30)
print(X_train.head())
print('-'*30)
print(len(y_train))
print('-'*30)
print(y_train.head())
print(len(X_test))
print('-'*30)
print(X_test.head())
print(len(y_test))
print('-'*30)
print(y_test.head())

print('-'*30)
list = y_test['smell'].values.tolist()
print('air     :' + str(list.count('air')))
print('tea :' + str(list.count('tea')))
print('clove :' + str(list.count('clove')))
print('coffee  :' + str(list.count('coffee')))

In [ ]:
#Feature scaling
sc_X = MinMaxScaler()
X_trainSc = sc_X.fit_transform(X_train)
X_testSc = sc_X.transform(X_test)

print(X_test)
print(X_testSc)

tmpdf = pd.DataFrame(X_trainSc)
print('-'*30)
print(tmpdf.describe())

# show histograms of the features
print('-'*30)
tmpdf.hist()
plt.show()


**MODEL TRAINING AND SAVING**

In [ ]:
# Nearest Neighbors classification decision boundaries
N_NEIGHBORS = 10

# Create color maps
cmap_light = ListedColormap(["pink", "lightblue", "yellow"])
cmap_bold = ["orange", "lightgreen", "red"]

# create K Neighbours Classifier and fit data.
classifier = KNeighborsClassifier(n_neighbors=N_NEIGHBORS, p=2,metric='euclidean', weights='uniform')
classifier.fit(X_trainSc, y_train['smell'].values.tolist())

# plot boundaires
_, ax = plt.subplots()
DecisionBoundaryDisplay.from_estimator(
    classifier,
    X_trainSc,
    cmap=cmap_light,
    ax=ax,
    response_method="predict",
    plot_method="pcolormesh",
    xlabel=X.columns[0],
    ylabel=X.columns[1],
    shading="auto",
)

# Plot training points
scatter = sns.scatterplot(
    x=X_trainSc[:, 0],
    y=X_trainSc[:, 1],
    hue=y_train['smell'],
    palette=cmap_bold,
    alpha=1.0,
    edgecolor="black",
)
scatter.set_xlim(left=-0.1, right=1.1)
scatter.set_ylim(bottom=-0.1, top=1.1);
    
plt.title(
    "Classification Boundary (k = %i, weights = '%s')" % (N_NEIGHBORS, 'uniform')
)

plt.show()

In [ ]:
# Predict the test set results
y_pred = classifier.predict(X_testSc)
print(y_pred)

In [ ]:
print('f1_score       : ' + str(f1_score(y_test, y_pred, average=None)))
print('accuracy_score : ' + str(accuracy_score(y_test, y_pred)))

In [ ]:
# Specify the directory where you want to save the model
directory = '/eNose-main/enose-api/'

# Make sure the directory exists, create it if it doesn't
if not os.path.exists(directory):
    os.makedirs(directory)

# Specify the full path including the filename
modelFilename = os.path.join(directory, 'enoseModel.pkl')
scFilename = os.path.join(directory, 'enoseSc.pkl')

# Save model in binary mode
with open(modelFilename, 'wb') as enosePickle:
    pickle.dump(classifier, enosePickle)

# Save fitted standardscalar in biniary mode
with open(scFilename, 'wb') as enosePickle:
    pickle.dump(sc_X, enosePickle)



**CHANGE labels IF NEW TRAINING ITEM IS ADDED IN BELOW CELL**

In [ ]:
# reload the model and scaler. Use them to run the smell detection using the same test data
# to see if we get the same results as before

# load the StandardScaler from disk and transform X_test
loaded_ss = pickle.load(open(scFilename, 'rb'))
print(loaded_ss)
X_testSs = loaded_ss.transform(X_test)
#X_testSs = X_test

# load the model from disk
loaded_model = pickle.load(open(modelFilename, 'rb'))
print(loaded_model)
y_pred = loaded_model.predict(X_testSs)

# the confusion matrix hould be exactly the same as the previous one
labels = ["air","isopropylAlcohol","sanitizer"]
cm = confusion_matrix(y_test, y_pred)
print(cm)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
disp.plot()
print('f1_score      : ' + str(f1_score(y_test, y_pred, average=None)))
print('accuracy_score: ' + str(accuracy_score(y_test, y_pred)))


**BELOW CELL SETS UP FLASK SERVER AND SHOWS LIVE PREDICTION**

**REPLACE THE MQTT SETTINGS WITH THE RIGHT ONE ACCORDING TO YOUR COMPUTER**

In [ ]:
from flask import Flask, render_template_string, jsonify
import random
from threading import Thread
import time
import paho.mqtt.client as mqtt
import pandas as pd
import json
import joblib  # For loading the model and scaler

# Define MQTT settings
MQTT_BROKER = "192.168.196.213"
MQTT_PORT = 1883
MQTT_TOPIC = "sensorData"
MQTT_USERNAME = "admin"
MQTT_PASSWORD = "admin" 

# File paths
MODEL_PATH = '/eNose-main/enose-api/enoseModel.pkl'  
SCALER_PATH = '/eNose-main/enose-api/enoseSc.pkl'

# Placeholder for incoming data
data_list = []
counter=[]

# Load your pre-trained ML model and scaler
model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# Flask app setup
app = Flask(__name__)
current_prediction = {'prediction': 'Unknown', 'image_path': '/static/images/unknown.jpg'}  # Default prediction

# HTML template as a string
html_template = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>E-nose prediction</title>
    <style>
        #image {
            max-width: 100%;
            height: auto;
        }
    </style>
    <script>
        async function updatePrediction() {
            const response = await fetch('/current-prediction');
            const data = await response.json();
            const prediction = data.prediction;
            const imagePath = data.image_path;
            document.getElementById('prediction').innerText = prediction;
            document.getElementById('prediction-image').src = imagePath;
        }

        window.onload = function() {
            updatePrediction();
            setInterval(updatePrediction, 2000); // Update every 2 seconds
        }
    </script>
</head>
<body>
    <h1>E-nose sensor prediction</h1>
    <p>Predicted: <span id="prediction">Loading...</span></p>
    <img id="prediction-image" src="/static/images/unknown.jpg" alt="Prediction Image">
</body>
</html>
'''

@app.route('/')
def index():
    return render_template_string(html_template)

@app.route('/current-prediction')
def current_prediction_route():
    global current_prediction
    return jsonify(current_prediction)

# MQTT message handler
def on_message(client, userdata, msg):
    global data_list
    global current_prediction
    try:
        # Decode the incoming message
        message = msg.payload.decode()
        
        # Convert JSON string to Python dictionary
        data = json.loads(message)
        
        # Extract datapoints from the received data
        datapoints = data.get("datapoints", [])
        
        # Print the received datapoints (for debugging)
        print("Received datapoints:", datapoints)
        
        # Extend data_list with the received datapoints
        data_list.extend(datapoints)
        
        # Convert list to DataFrame
        df = pd.DataFrame(data_list, columns=['index', 'timestamp', 'temperature', 'pressure', 'humidity', 'gas_resistance'])
        
        # Predict using the latest data point
        latest_data = df.iloc[[-1]]  # Use the latest row for prediction
        
        # Apply the scaler to the latest data point
        scaled_data = scaler.transform(latest_data.drop(['index', 'timestamp', 'temperature', 'pressure'], axis=1))  # Remove non-numeric columns
        
        # Make predictions using the model
        prediction = model.predict(scaled_data)
        
        # Print the prediction (or handle it as needed)
        print(f"Prediction: {prediction}")

        counter.append(prediction[0])
        
        # Update the current_prediction based on the prediction
        if len(counter) > 3:
            if counter[-1] == counter[-2] == counter[-3]:
                if counter[-1] == 'coffee':
                    current_prediction['prediction'] = 'coffee'
                    current_prediction['image_path'] = '/static/images/coffee.jpg'
                elif counter[-1] == "air":
                    current_prediction['prediction'] = 'air'
                    current_prediction['image_path'] = '/static/images/air.jpg'
                elif counter[-1] == "sanitizer":
                    current_prediction['prediction'] = 'sanitizer'
                    current_prediction['image_path'] = '/static/images/sanitizer.jpg'
                elif counter[-1] == "isopropylAlcohol":
                    current_prediction['prediction'] = 'isopropylAlcohol'
                    current_prediction['image_path'] = '/static/images/isopropylAlcohol.jpg'
            else:
                current_prediction['prediction'] = 'Unknown'
                current_prediction['image_path'] = '/static/images/unknown.jpg'
        else:
            current_prediction['prediction'] = 'Unknown'
            current_prediction['image_path'] = '/static/images/unknown.jpg'
        
    except Exception as e:
        print(f"Error processing message: {e}")

# Set up MQTT client with authentication
client = mqtt.Client()
client.username_pw_set(MQTT_USERNAME, MQTT_PASSWORD)  # Set username and password
client.on_message = on_message

# Connect to the MQTT broker
client.connect(MQTT_BROKER, MQTT_PORT, 60)

# Subscribe to the MQTT topic
client.subscribe(MQTT_TOPIC)

# Start the MQTT client loop to begin listening for messages
def start_mqtt_client():
    client.loop_forever()

# Start the Flask server in another background thread
def run_app():
    app.run(debug=True, use_reloader=False, port=5001)

# Start the MQTT client loop in another background thread
mqtt_thread = Thread(target=start_mqtt_client)
mqtt_thread.start()

# Start the Flask server in another background thread
flask_thread = Thread(target=run_app)
flask_thread.start()
